In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, mean_absolute_error, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.impute import SimpleImputer
from torch.distributions import Normal, kl_divergence
import warnings
warnings.filterwarnings('ignore')

In [25]:
# Байесовский линейный слой
class BayesianLinear(nn.Module):
    def __init__(self, in_features, out_features, prior_sigma=1.0):
        super(BayesianLinear, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Параметры апостериорного распределения весов
        self.weight_mu = nn.Parameter(torch.Tensor(out_features, in_features).normal_(0, 0.1))
        self.weight_rho = nn.Parameter(torch.Tensor(out_features, in_features).normal_(-3, 0.1))
        
        # Параметры апостериорного распределения смещений
        self.bias_mu = nn.Parameter(torch.Tensor(out_features).normal_(0, 0.1))
        self.bias_rho = nn.Parameter(torch.Tensor(out_features).normal_(-3, 0.1))
        
        # Априорное распределение
        self.prior_sigma = prior_sigma
        # self.register_buffer('weight_prior', Normal(0, prior_sigma))
        # self.register_buffer('bias_prior', Normal(0, prior_sigma))
        
    @property
    def weight_sigma(self):
        return torch.log1p(torch.exp(self.weight_rho))
    
    @property
    def bias_sigma(self):
        return torch.log1p(torch.exp(self.bias_rho))
    
    def forward(self, x, sample=True):
        if self.training or sample:
            # Сэмплирование весов во время обучения
            weight_epsilon = torch.randn_like(self.weight_mu)
            bias_epsilon = torch.randn_like(self.bias_mu)
            
            weight = self.weight_mu + self.weight_sigma * weight_epsilon
            bias = self.bias_mu + self.bias_sigma * bias_epsilon
        else:
            # Использование средних значений во время инференса
            weight = self.weight_mu
            bias = self.bias_mu
            
        return F.linear(x, weight, bias)
    
    def kl_loss(self):
        # Создаем априорные распределения на лету
        weight_prior = Normal(0, self.prior_sigma)
        bias_prior = Normal(0, self.prior_sigma)
        
        # Апостериорные распределения
        weight_dist = Normal(self.weight_mu, self.weight_sigma)
        bias_dist = Normal(self.bias_mu, self.bias_sigma)
        
        kl = kl_divergence(weight_dist, weight_prior).sum()
        kl += kl_divergence(bias_dist, bias_prior).sum()
        
        return kl

In [27]:
# Байесовская нейронная сеть
class BayesianInsuranceNet(nn.Module):
    def __init__(self, input_size, prior_sigma=1.0):
        super(BayesianInsuranceNet, self).__init__()
        
        self.layer1 = BayesianLinear(input_size, 72, prior_sigma)
        self.layer2 = BayesianLinear(72, 64, prior_sigma)
        self.layer3 = BayesianLinear(64, 54, prior_sigma)
        self.layer4 = BayesianLinear(54, 36, prior_sigma)
        self.layer5 = BayesianLinear(36, 24, prior_sigma)
        self.layer6 = BayesianLinear(24, 1, prior_sigma)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x, sample=True):
        x = self.relu(self.layer1(x, sample))
        x = self.relu(self.layer2(x, sample))
        x = self.relu(self.layer3(x, sample))
        x = self.relu(self.layer4(x, sample))
        x = self.relu(self.layer5(x, sample))
        x = self.sigmoid(self.layer6(x, sample))
        return x
    
    def kl_loss(self):
        kl = 0
        for module in self.modules():
            if isinstance(module, BayesianLinear):
                kl += module.kl_loss()
        return kl
# Функция потерь для байесовской сети
def bayesian_loss(prediction, target, model, kl_weight=0.1):
    likelihood_loss = F.binary_cross_entropy(prediction, target)
    kl_loss = model.kl_loss()
    total_loss = likelihood_loss + kl_weight * kl_loss
    return total_loss, likelihood_loss, kl_loss


In [28]:
# Обучение байесовской модели
def train_bayesian_model(model, X_train, y_train, epochs=350, kl_weight=0.1):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Прямой проход с сэмплированием весов
        outputs = model(X_train, sample=True)
        total_loss, likelihood_loss, kl_loss = bayesian_loss(outputs, y_train, model, kl_weight)
        
        total_loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Total Loss: {total_loss.item():.4f}, '
                  f'Likelihood: {likelihood_loss.item():.4f}, KL: {kl_loss.item():.4f}')

In [29]:
# Предсказание с учетом uncertainty
def predict_with_uncertainty(model, X, num_samples=50):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for _ in range(num_samples):
            pred = model(X, sample=True)
            predictions.append(pred.numpy())
    
    predictions = np.array(predictions)
    mean_prediction = predictions.mean(axis=0)
    std_prediction = predictions.std(axis=0)
    
    return mean_prediction, std_prediction

In [30]:
print("Загрузка данных...")
data = pd.read_csv('D:/my_ML/diploma_polytech/data/raw/vehicle_ins_data_1.csv', sep=";", index_col=False, low_memory=False)

# Создание целевой переменной: 1 если N_claims_year > 1, иначе 0
data['claim_prob'] = (data['N_claims_year'] > 1).astype(int)

# Предобработка данных
# Удаление ненужных столбцов, но НЕ УДАЛЯЕМ Date_start_contract
cols_to_drop = ['ID', 'Date_last_renewal', 'Date_next_renewal', 
                'Date_lapse', 'N_claims_year', 'Cost_claims_year', 'N_claims_history', 
                'R_Claims_history']
data = data.drop(columns=cols_to_drop)

# Преобразуем Date_start_contract в datetime ДО любой обработки
data['Date_start_contract'] = pd.to_datetime(data['Date_start_contract'], errors='coerce')

# Обработка датовых колонок
date_columns = ['Date_birth', 'Date_driving_licence']
existing_date_columns = [col for col in date_columns if col in data.columns]

print("Существующие датовые колонки для обработки:", existing_date_columns)

if not existing_date_columns:
    print("Предупреждение: Датовые колонки не найдены в DataFrame.")
else:
    for col in existing_date_columns:
        data[col] = pd.to_datetime(data[col], format='%d/%m/%Y', errors='coerce')

# Извлечение числовых признаков из дат
reference_date = pd.to_datetime('2019-12-31')

if 'Date_birth' in data.columns:
    data['Age'] = (reference_date - data['Date_birth']).dt.days // 365

if 'Date_driving_licence' in data.columns:
    data['Driving_experience'] = (reference_date - data['Date_driving_licence']).dt.days // 365

# Удаление исходных датовых колонок
data = data.drop(columns=date_columns, errors='ignore')


Загрузка данных...
Существующие датовые колонки для обработки: ['Date_birth', 'Date_driving_licence']


In [31]:

# Обработка категориальных переменных
categorical_cols = ['Type_fuel']
label_encoders = {}
for col in categorical_cols:
    if col in data.columns:
        le = LabelEncoder()
        data[col] = le.fit_transform(data[col].astype(str))
        label_encoders[col] = le

# Убедиться, что все данные числовые
data = data.apply(pd.to_numeric, errors='coerce')

# Обработка пропущенных значений
imputer = SimpleImputer(strategy='median')
data_imputed = imputer.fit_transform(data)
data = pd.DataFrame(data_imputed, columns=data.columns)

print(f"Размер данных после предобработки: {data.shape}")
print(f"Баланс классов: {data['claim_prob'].value_counts()}")

# Разделение на признаки и целевую переменную
X = data.drop(columns=['claim_prob'])
y = data['claim_prob']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Масштабирование признаков
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Преобразование в тензоры PyTorch
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

print(f"Размер тренировочной выборки: {X_train_tensor.shape}")
print(f"Размер тестовой выборки: {X_test_tensor.shape}")

# Инициализация байесовской модели
input_size = X_train_tensor.shape[1]
bayesian_model = BayesianInsuranceNet(input_size, prior_sigma=1.0)

print(f"\nИнициализирована байесовская нейронная сеть с {input_size} входными признаками")

Размер данных после предобработки: (105555, 23)
Баланс классов: claim_prob
0.0    95448
1.0    10107
Name: count, dtype: int64
Размер тренировочной выборки: torch.Size([84444, 22])
Размер тестовой выборки: torch.Size([21111, 22])

Инициализирована байесовская нейронная сеть с 22 входными признаками


In [32]:
print("\nНачало обучения байесовской нейронной сети...")
train_bayesian_model(bayesian_model, X_train_tensor, y_train_tensor, epochs=500, kl_weight=0.1)



Начало обучения байесовской нейронной сети...
Epoch [10/500], Total Loss: 3209.6406, Likelihood: 0.7094, KL: 32089.3105
Epoch [20/500], Total Loss: 3196.3696, Likelihood: 0.6568, KL: 31957.1289
Epoch [30/500], Total Loss: 3183.2568, Likelihood: 0.6359, KL: 31826.2070
Epoch [40/500], Total Loss: 3170.2754, Likelihood: 0.6321, KL: 31696.4336
Epoch [50/500], Total Loss: 3157.4128, Likelihood: 0.6494, KL: 31567.6328
Epoch [60/500], Total Loss: 3144.6206, Likelihood: 0.6520, KL: 31439.6836
Epoch [70/500], Total Loss: 3131.8677, Likelihood: 0.6230, KL: 31312.4453
Epoch [80/500], Total Loss: 3119.1782, Likelihood: 0.5970, KL: 31185.8125
Epoch [90/500], Total Loss: 3106.5012, Likelihood: 0.5313, KL: 31059.6992
Epoch [100/500], Total Loss: 3093.9995, Likelihood: 0.5962, KL: 30934.0332
Epoch [110/500], Total Loss: 3081.4648, Likelihood: 0.5889, KL: 30808.7598
Epoch [120/500], Total Loss: 3068.9290, Likelihood: 0.5461, KL: 30683.8281
Epoch [130/500], Total Loss: 3056.4890, Likelihood: 0.5694, KL

In [34]:
# Оценка модели с учетом uncertainty
print("\nОценка байесовской модели...")
#y_pred_mean, y_pred_std = predict_with_uncertainty(bayesian_model, X_test_tensor, num_samples=100)
bayesian_model.eval()
predictions = []
    
with torch.no_grad():
    for _ in range(len(X_test_tensor)):
        pred = bayesian_model(X_test_tensor, sample=True)
        predictions.append(pred.numpy())
    
predictions = np.array(predictions)
# Бинарные предсказания на основе среднего
#y_pred_binary = (y_pred_mean > 0.5).astype(int)

# Метрики
roc_auc = roc_auc_score(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
#mape_score = mean_absolute_percentage_error(y_test, y_pred_mean)
#accuracy = accuracy_score(y_test, y_pred_binary)
#precision = precision_score(y_test, y_pred_binary, zero_division=0)
#recall = recall_score(y_test, y_pred_binary, zero_division=0)
#f1 = f1_score(y_test, y_pred_binary, zero_division=0)

print("\n" + "="*50)
print("РЕЗУЛЬТАТЫ БАЙЕСОВСКОЙ НЕЙРОННОЙ СЕТИ")
print("="*50)
print(f'ROC-AUC: {roc_auc:.4f}')
print(f'MAE: {mae:.4f}')
#print(f'MAPE: {mape_score:.4f}%')
# print(f'Accuracy: {accuracy:.4f}')
# print(f'Precision: {precision:.4f}')
# print(f'Recall: {recall:.4f}')
#print(f'F1-Score: {f1:.4f}')



Оценка байесовской модели...


KeyboardInterrupt: 